# Import libraries

In [77]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from helpers import parse_variables
import scipy.stats as stats

# Extracting simulated data from rstudio

In [78]:
os.makedirs(f"data/genotype", exist_ok = True)

In [79]:
dict = parse_variables('geno_simulation.txt')
G = int(dict['G'])
L = int(dict['L'])
c = int(dict['c'])
k = int(dict['k'])
M = float(dict['M'])


# Thresholds
very_rare_threshold_L = float(dict['very_rare_threshold_L'])
very_rare_threshold_H = float(dict['very_rare_threshold_H'])

rare_threshold_L = float(dict['rare_threshold_L'])
rare_threshold_H = float(dict['rare_threshold_H'])

common_threshold_L = float(dict['common_threshold_L'])
common_threshold_H = float(dict['common_threshold_H'])

file = f"data/raw/simulated_genotypes_G{G}_L{L}_c{c}_k{k}_M{M}.csv"

In [80]:
path_simulated_file = "./"+ file

In [81]:
number_of_loci = G*L
number_of_loci

20000

In [82]:
number_of_individuals = c*k*k
number_of_individuals

20000

In [ ]:
simulated_loci= pd.read_csv(path_simulated_file)

# Go from loci to genotype

In [ ]:
# Function to pair SNPs and summarize genotype
def summarize_genotypes(df):
    summarized_genotypes = {}
    # Iterate over pairs of columns
    for i in range(1, df.shape[1], 2):
        pair_sum = df.iloc[:, i-1] + df.iloc[:, i]
        # Apply the genotype summarization logic
        summarized_genotypes[f'G{i//2 + 1}'] = np.where(pair_sum == 2, 2, pair_sum)
    return pd.DataFrame(summarized_genotypes)

# Apply the function to the sample DataFrame
simulated_genotype = summarize_genotypes(simulated_loci)
columns_to_drop  = simulated_genotype.columns[simulated_genotype.nunique() == 1] # If double columns delete it 
simulated_genotype = simulated_genotype.drop(columns=columns_to_drop)

In [ ]:
simulated_genotype

# Switch genotypes of AF > 0.5

In [ ]:
# calculate when AF is > 0.5 and change the genotype
# Initialize a dictionary to store allele frequencies
allele_frequencies = {}

# Calculate allele frequencies for each SNP column
for snp in simulated_genotype.columns:
    total_alleles = 2 * len(simulated_genotype[snp])  # Total number of alleles (2 alleles per sample)
    minor_allele_count = (2 * simulated_genotype[snp].value_counts().get(0, 0)) + simulated_genotype[snp].value_counts().get(1, 0)
    allele_frequency = minor_allele_count / total_alleles
    allele_frequencies[snp] = allele_frequency

In [ ]:
temp = simulated_genotype.T
temp['AFs'] = allele_frequencies

In [ ]:
temp

In [ ]:
# Function to flip 0s to 2s and 2s to 0s
def flip_genotypes(row):
    if row['AFs'] > 0.5:
        # Apply transformation for the condition
        row[:-1] = row[:-1].replace({0: 2, 2: 0})
        row['AFs'] = 1 - row['AFs']  # Adjust allele frequency
    return row

# Apply the function across the DataFrame, row-wise
df_transformed = temp.apply(flip_genotypes, axis=1)

In [ ]:
df_transformed

# Recheck if there are dupplicates again

In [ ]:
simulated_genotype = df_transformed.drop('AFs', axis=1).T
columns_to_drop  = simulated_genotype.columns[simulated_genotype.nunique() == 1] # If double columns delete it 
simulated_genotype = simulated_genotype.drop(columns=columns_to_drop)
simulated_genotype

In [ ]:
def hwe_test(genotypes):
    """
    Perform a chi-square test for Hardy-Weinberg Equilibrium.
    Returns p-value of the test.
    """
    # Count genotype frequencies
    obs_aa = sum(genotypes == 0)
    obs_ab = sum(genotypes == 1)
    obs_bb = sum(genotypes == 2)
    total = obs_aa + obs_ab + obs_bb

    # Calculate allele frequencies
    p = (2 * obs_aa + obs_ab) / (2 * total)
    q = 1 - p

    # Expected genotype frequencies
    exp_aa = total * p**2
    exp_ab = total * 2 * p * q
    exp_bb = total * q**2

    # Avoid zero expected counts by using a small value (e.g., 1e-10)
    expected = np.array([exp_aa, exp_ab, exp_bb])
    expected[expected == 0] = 1e-10

    observed = np.array([obs_aa, obs_ab, obs_bb])
    
    # Chi-square test
    chi2, p_value = stats.chisquare(f_obs=observed, f_exp=expected)

    return p_value

In [ ]:
def contains_all_genotypes(series, genotypes={0.0, 1.0, 2.0}):
    return genotypes.issubset(series.unique())

simulated_genotype = simulated_genotype[[col for col in simulated_genotype.columns if contains_all_genotypes(simulated_genotype[col])]]
simulated_genotype

# HWE

In [ ]:
# Threshold for HWE p-value
threshold = 0.05

number_of_populations = k*k
labels_pop = []
for i in range(number_of_populations):
    labels_pop += [f"pop {i+1}"]*c

In [ ]:
simulated_genotype["populations"] = labels_pop

In [ ]:
hwe_dfs = []
unique_pops = list(set(labels_pop))
for pop in unique_pops:
    temp_pop = simulated_genotype[simulated_genotype["populations"] == pop]
    temp_pop = temp_pop.drop('populations', axis=1)
    # List to hold columns in HWE
    hwe_columns = []
    
    for column in temp_pop.columns:
        p_value = hwe_test(temp_pop[column].values)
        if p_value > threshold:
            hwe_columns.append(column)
    cols_in_hwe = temp_pop[hwe_columns]
    hwe_dfs.append(cols_in_hwe)

In [ ]:
combined_df = pd.concat(hwe_dfs, axis=0)

In [ ]:
combined_df

In [ ]:
sorted_df = combined_df.sort_index()
# Drop columns with any NaN values
sorted_df.fillna(2.0, inplace=True)
simulated_genotype = sorted_df

In [ ]:
sorted_df

In [ ]:
simulated_genotype

In [ ]:
#simulated_genotype
#temp_pop = simulated_genotype.drop('populations', axis=1)
#hwe_columns = []
#    
#for column in temp_pop.columns:
#    p_value = hwe_test(temp_pop[column].values)
#    if p_value > threshold:
#        hwe_columns.append(column)
#cols_in_hwe = temp_pop[hwe_columns]

In [62]:
#cols_in_hwe

# Recalculate AFs

In [63]:
# calculate when AF is > 0.5 and change the genotype
# Initialize a dictionary to store allele frequencies
allele_frequencies = {}

# Calculate allele frequencies for each SNP column
for snp in simulated_genotype.columns:
    total_alleles = 2 * len(simulated_genotype[snp])  # Total number of alleles (2 alleles per sample)
    minor_allele_count = (2 * simulated_genotype[snp].value_counts().get(0, 0)) + simulated_genotype[snp].value_counts().get(1, 0)
    allele_frequency = minor_allele_count / total_alleles
    allele_frequencies[snp] = allele_frequency

In [64]:
temp = simulated_genotype.T
temp['AFs'] = allele_frequencies
AFs = temp[['AFs']]

In [65]:
AFs

,AFs
G7,0.107450
G10,0.000900
G14,0.019875
G16,0.026025
G29,0.017100
...,...
G312,0.005825
G327,0.005825
G407,0.036450
G422,0.004050


# Save complete genotype

In [66]:
simulated_genotype.to_pickle(f"data/genotype/simulated_complete_genotypes_AF_0_0.5_G{G}_L{L}_c{c}_k{k}_M{M}.pkl")

In [67]:
AFs.to_pickle(f"data/genotype/simulated_complete_frequencies_AF_0_0.5_G{G}_L{L}_c{c}_k{k}_M{M}.pkl")

# Divide into extra rare, rare, common 

In [68]:
very_rare = temp[(temp['AFs'] > very_rare_threshold_L) & (temp['AFs'] <= very_rare_threshold_H)]
rare = temp[(temp['AFs'] > rare_threshold_L) & (temp['AFs'] <= rare_threshold_H)]
common = temp[(temp['AFs'] > common_threshold_L) & (temp['AFs'] <= common_threshold_H)]

very_rare['snps'] = very_rare.index + '_AF_' + very_rare['AFs'].astype(str)
very_rare.set_index('snps', inplace=True)
very_rare_to_save = very_rare.drop('AFs', axis=1).T
very_rare_afs = very_rare[['AFs']]

rare['snps'] = rare.index + '_AF_' + rare['AFs'].astype(str)
rare.set_index('snps', inplace=True)
rare_to_save = rare.drop('AFs', axis=1).T
rare_afs = rare[['AFs']]

common['snps'] = common.index + '_AF_' + common['AFs'].astype(str)
common.set_index('snps', inplace=True)
common_to_save = common.drop('AFs', axis=1).T
common_afs = common[['AFs']]

/tmp/ipykernel_3347/1320502019.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  very_rare['snps'] = very_rare.index + '_AF_' + very_rare['AFs'].astype(str)
/tmp/ipykernel_3347/1320502019.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rare['snps'] = rare.index + '_AF_' + rare['AFs'].astype(str)
/tmp/ipykernel_3347/1320502019.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

In [69]:
very_rare_to_save

snps,G10_AF_0.0009,G14_AF_0.019875,G16_AF_0.026025,G29_AF_0.0171,G30_AF_0.0177,G51_AF_0.015125,G57_AF_0.020925,G72_AF_0.003425,G91_AF_0.003425,G111_AF_0.028875,...,G844_AF_0.019875,G845_AF_0.03625,G846_AF_0.03055,G343_AF_0.024,G306_AF_0.012675,G312_AF_0.005825,G327_AF_0.005825,G407_AF_0.03645,G422_AF_0.00405,G294_AF_0.001625
0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
1,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
2,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
3,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
4,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
19996,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
19997,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
19998,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0


In [70]:
very_rare_to_save.to_pickle(f"data/genotype/simulated_veryrare_genotype_AF_{very_rare_threshold_L}_{very_rare_threshold_H}_simulated_geno_G{G}_L{L}_c{c}_k{k}_M{M}.pkl")
rare_to_save.to_pickle(f"data/genotype/simulated_rare_genotype_AF_{rare_threshold_L}_{rare_threshold_H}_simulated_geno_G{G}_L{L}_c{c}_k{k}_M{M}.pkl")
common_to_save.to_pickle(f"data/genotype/simulated_common_genotype_AF_{common_threshold_L}_{common_threshold_H}_simulated_geno_G{G}_L{L}_c{c}_k{k}_M{M}.pkl")

In [71]:
very_rare_afs.to_pickle(f"data/genotype/simulated_veryrare_frequencies_AF_{very_rare_threshold_L}_{very_rare_threshold_H}_simulated_geno_G{G}_L{L}_c{c}_k{k}_M{M}.pkl")
rare_afs.to_pickle(f"data/genotype/simulated_rare_frequencies_AF_{rare_threshold_L}_{rare_threshold_H}_simulated_geno_G{G}_L{L}_c{c}_k{k}_M{M}.pkl")
common_afs.to_pickle(f"data/genotype/simulated_common_frequencies_AF_{common_threshold_L}_{common_threshold_H}_simulated_geno_G{G}_L{L}_c{c}_k{k}_M{M}.pkl")

In [72]:
common_afs

,AFs
snps,
G939_AF_0.23945,0.23945


In [73]:
rare_afs

,AFs
snps,
G7_AF_0.10745,0.107450
G98_AF_0.05195,0.051950
G178_AF_0.058975,0.058975
G272_AF_0.0889,0.088900
G328_AF_0.084175,0.084175
G370_AF_0.11005,0.110050
G435_AF_0.0649,0.064900
G535_AF_0.106975,0.106975
G719_AF_0.0899,0.089900


In [74]:
very_rare_afs

,AFs
snps,
G10_AF_0.0009,0.000900
G14_AF_0.019875,0.019875
G16_AF_0.026025,0.026025
G29_AF_0.0171,0.017100
G30_AF_0.0177,0.017700
...,...
G312_AF_0.005825,0.005825
G327_AF_0.005825,0.005825
G407_AF_0.03645,0.036450
